# Website Trivia Generator

## Week 1 Community Contribution - dc_dalin

This tool takes any website URL and generates interactive trivia questions based on its content.

**Use Cases:**
- Educational content creation
- Team building activities  
- Learning comprehension checks
- Content engagement tools

**Features:**
- Generates multiple choice and true/false questions
- Supports both OpenAI and Ollama models
- Creates questions with varying difficulty levels
- Clean markdown output

In [ ]:
import os
import sys
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

# Import scraper from parent week1 directory
sys.path.append('../..')
from scraper import fetch_website_contents

## Setup Environment

Load API keys and configure clients for both OpenAI and Ollama.

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-'):
    print("OpenAI API key found!")
else:
    print("No OpenAI API key found - will use Ollama only")

# Initialize OpenAI client
openai_client = OpenAI()

# Initialize Ollama client (running locally)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

## Configuration

Set the models and parameters for trivia generation.

In [ ]:
# Model selection
MODEL_GPT = 'gpt-4o-mini'
MODEL_OLLAMA = 'llama3.2'

# Number of questions to generate
NUM_QUESTIONS = 5

## Prompt Engineering

Crafting the system and user prompts for optimal trivia generation.

In [ ]:
system_prompt = f"""
You are a trivia question generator. Your task is to create engaging, accurate trivia questions based on website content.

Generate exactly {NUM_QUESTIONS} questions in the following format:

**Question 1:** [Question text]
A) [Option A]
B) [Option B]
C) [Option C]
D) [Option D]

**Correct Answer:** [Letter]

Guidelines:
- Mix multiple choice and true/false questions
- Vary difficulty levels (easy, medium, hard)
- Focus on key facts and interesting details
- Make incorrect options plausible but clearly wrong
- Avoid questions about navigation or website structure
- Respond in markdown format
"""

In [ ]:
def create_user_prompt(content):
    return f"""
Based on the following website content, generate {NUM_QUESTIONS} trivia questions:

{content}

Remember to:
- Create questions about the main topics and key information
- Include the correct answer for each question
- Make the questions educational and engaging
"""

## Core Functions

Functions to generate trivia using OpenAI or Ollama.

In [ ]:
def generate_trivia_openai(url):
    """
    Generate trivia questions using OpenAI's GPT model.
    """
    print(f"Fetching content from {url}...")
    content = fetch_website_contents(url)
    
    print("Generating trivia questions with OpenAI...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": create_user_prompt(content)}
    ]
    
    response = openai_client.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def generate_trivia_ollama(url):
    """
    Generate trivia questions using Ollama's local model.
    """
    print(f"Fetching content from {url}...")
    content = fetch_website_contents(url)
    
    print("Generating trivia questions with Ollama...")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": create_user_prompt(content)}
    ]
    
    response = ollama_client.chat.completions.create(
        model=MODEL_OLLAMA,
        messages=messages,
        temperature=0.7
    )
    
    return response.choices[0].message.content

In [ ]:
def display_trivia(url, use_ollama=False):
    """
    Generate and display trivia questions in markdown format.
    """
    try:
        if use_ollama:
            trivia = generate_trivia_ollama(url)
            print("\n" + "="*60)
            print(f"TRIVIA GENERATED WITH OLLAMA ({MODEL_OLLAMA})")
            print("="*60)
        else:
            trivia = generate_trivia_openai(url)
            print("\n" + "="*60)
            print(f"TRIVIA GENERATED WITH OPENAI ({MODEL_GPT})")
            print("="*60)
        
        display(Markdown(trivia))
        
    except Exception as e:
        print(f"Error generating trivia: {e}")
        print("Make sure Ollama is running if using local models.")

## Examples

Try generating trivia from different websites!

### Example 1: Using OpenAI (GPT-4o-mini)

In [ ]:
# Generate trivia about Anthropic using OpenAI
display_trivia("https://anthropic.com", use_ollama=False)

### Example 2: Using Ollama (Local Model)

Make sure Ollama is running locally with: `ollama run llama3.2`

In [ ]:
# Generate trivia about BBC News using Ollama
display_trivia("https://bbc.com", use_ollama=True)

### Try Your Own URL!

Enter any website URL below to generate custom trivia questions.

In [ ]:
# Try your own URL here
custom_url = "https://edwarddonner.com"
use_ollama = False  # Set to True to use Ollama instead of OpenAI

display_trivia(custom_url, use_ollama=use_ollama)

## Business Applications

This trivia generator demonstrates several Week 1 concepts:

1. **Web Scraping**: Extracting content from any URL
2. **Prompt Engineering**: Crafting effective system and user prompts
3. **Multi-Model Support**: Working with both cloud (OpenAI) and local (Ollama) models
4. **Practical Application**: Creating engaging educational content

**Real-world use cases:**
- E-learning platforms: Auto-generate quiz questions from course materials
- Content marketing: Create engagement tools for blog posts
- Corporate training: Generate assessment questions from policy documents
- News sites: Build interactive quizzes from articles
- Museums/Tourism: Create trivia from exhibit descriptions

## Ideas for Extension

- Add difficulty level selection
- Generate answer explanations
- Create themed trivia (history, science, tech)
- Export to JSON for use in apps
- Add scoring system
- Support multiple URLs for comprehensive quizzes
- Generate image-based questions if website has photos